In [1]:
import cv2
import numpy as np
from pathlib import Path

from hfnet.settings import EXPER_PATH
from notebooks.utils import plot_images, plot_matches, add_frame

import tensorflow as tf
print(tf.__version__)
from tensorflow.python.saved_model import tag_constants
tf.contrib.resampler  # import C++ op
from os import listdir
from os.path import isfile, join
import os, sys


%load_ext autoreload
%autoreload 2
%matplotlib inline

/home/pang/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pang/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pang/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/pang/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWa

1.12.0


# Create HF-Net model for inference

In [2]:
class HFNet:
    def __init__(self, model_path, outputs):
        self.session = tf.Session()
        self.image_ph = tf.placeholder(tf.float32, shape=(None, None, 3))

        net_input = tf.image.rgb_to_grayscale(self.image_ph[None])
        tf.saved_model.loader.load(
            self.session, [tag_constants.SERVING], str(model_path),
            clear_devices=True,
            input_map={'image:0': net_input})

        graph = tf.get_default_graph()
        self.outputs = {n: graph.get_tensor_by_name(n+':0')[0] for n in outputs}
        self.nms_radius_op = graph.get_tensor_by_name('pred/simple_nms/radius:0')
        self.num_keypoints_op = graph.get_tensor_by_name('pred/top_k_keypoints/k:0')
        
    def inference(self, image, nms_radius=4, num_keypoints=1000):
        inputs = {
            self.image_ph: image[..., ::-1].astype(np.float),
            self.nms_radius_op: nms_radius,
            self.num_keypoints_op: num_keypoints,
        }
        return self.session.run(self.outputs, feed_dict=inputs)

model_path = Path(EXPER_PATH, 'saved_models/hfnet')
outputs = ['global_descriptor', 'keypoints', 'local_descriptors']
hfnet = HFNet(model_path, outputs)

[10/23/2019 14:05:23 INFO] Restoring parameters from /home/pang/software/hfnet_venv/hfnet_exp/saved_models/hfnet/variables/variables


# Compute global descriptors and local features for query and database

In [3]:
# inference images
dataset_folder = "/home/pang/data/dataset/iros2019slam/office/office-1-5/"
image_list_folder = dataset_folder + "fisheye1"
kp_record_folder = dataset_folder + "fisheye1_hfnet_kps"
local_des_record_folder = dataset_folder + "fisheye1_hfnet_local_des"
global_des_record_folder = dataset_folder + "fisheye1_hfnet_global_des"
if not os.path.exists(kp_record_folder):
    os.mkdir(kp_record_folder)
if not os.path.exists(local_des_record_folder):
    os.mkdir(local_des_record_folder)
if not os.path.exists(global_des_record_folder):
    os.mkdir(global_des_record_folder)


# query = hfnet.inference(image_query)
# # print (query)
# # print ((query['global_descriptor']))
# print ((query['global_descriptor']).shape)
# print ((query['keypoints']).shape)
# print (len(query['keypoints']))
# print ((query['local_descriptors']).shape)
# print (len(query['local_descriptors']))

image_files = [f[:-4] for f in listdir(image_list_folder) if isfile(join(image_list_folder, f))]
image_files.sort(key=lambda f: int(f))

total = len(image_files)
cnt = 0
# print (image_files)
for image in image_files:
    image_name = image_list_folder + "/" + image + ".png";
    print (image_name)
    query_image = cv2.imread(image_name)[:, :, ::-1]
    print (query_image.shape)
    kp_file = kp_record_folder + "/" + image + ".txt"
    local_des_file = local_des_record_folder + "/" + image + ".txt"
    global_des_file = global_des_record_folder + "/" + image + ".txt"
    
    kp_ofs = open(kp_file, 'w')
    local_des_ofs = open(local_des_file, 'w')
    global_des_ofs = open(global_des_file, 'w')
    
    # inference 
    query_info = hfnet.inference(query_image)
    
    # write kp
    for kp in query_info['keypoints']:
        kp_ofs.write(str(kp[0]) + " " + str(kp[1]) + "\n")
#         print (type(kp))
    kp_ofs.close()
        
        
    # write global des
    for global_des in query_info['global_descriptor']:
        global_des_ofs.write(str(global_des) + "\n")
    
    global_des_ofs.close()
    
            
    # write local des
    for local_des in query_info['local_descriptors']:
        for i in local_des:
            local_des_ofs.write(str(i) + " ")
        local_des_ofs.write("\n")
    local_des_ofs.close()
    
    cnt  = cnt + 1
    process = str(cnt) + " / " + str(total)
    print (process)
        


/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116808865494.png
(800, 848, 3)
1 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116808898828.png
(800, 848, 3)
2 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116808932161.png
(800, 848, 3)
3 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116808965495.png
(800, 848, 3)
4 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116808998828.png
(800, 848, 3)
5 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116809032161.png
(800, 848, 3)
6 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116809065495.png
(800, 848, 3)
7 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116809098828.png
(800, 848, 3)
8 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116809132162.png
(800, 848, 3)
9 / 1591
/home/pang/data/dataset/iros

76 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116811398694.png
(800, 848, 3)
77 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116811432027.png
(800, 848, 3)
78 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116811465361.png
(800, 848, 3)
79 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116811498694.png
(800, 848, 3)
80 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116811532027.png
(800, 848, 3)
81 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116811565361.png
(800, 848, 3)
82 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116811598694.png
(800, 848, 3)
83 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116811632028.png
(800, 848, 3)
84 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116811665423.png
(800, 848, 3)
85 / 1591
/home/pan

151 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116813898556.png
(800, 848, 3)
152 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116813931889.png
(800, 848, 3)
153 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116813965223.png
(800, 848, 3)
154 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116813998556.png
(800, 848, 3)
155 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116814031890.png
(800, 848, 3)
156 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116814065223.png
(800, 848, 3)
157 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116814098556.png
(800, 848, 3)
158 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116814131890.png
(800, 848, 3)
159 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116814165119.png
(800, 848, 3)
160 / 1591

226 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116816398449.png
(800, 848, 3)
227 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116816431782.png
(800, 848, 3)
228 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116816465116.png
(800, 848, 3)
229 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116816498449.png
(800, 848, 3)
230 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116816531782.png
(800, 848, 3)
231 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116816565115.png
(800, 848, 3)
232 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116816598449.png
(800, 848, 3)
233 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116816631783.png
(800, 848, 3)
234 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116816665046.png
(800, 848, 3)
235 / 1591

301 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116818898333.png
(800, 848, 3)
302 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116818931666.png
(800, 848, 3)
303 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116818964999.png
(800, 848, 3)
304 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116818998333.png
(800, 848, 3)
305 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116819031666.png
(800, 848, 3)
306 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116819065000.png
(800, 848, 3)
307 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116819098333.png
(800, 848, 3)
308 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116819131666.png
(800, 848, 3)
309 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116819165000.png
(800, 848, 3)
310 / 1591

...

1273 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116851297003.png
(800, 848, 3)
1274 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116851330336.png
(800, 848, 3)
1275 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116851363669.png
(800, 848, 3)
1276 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116851397003.png
(800, 848, 3)
1277 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116851430336.png
(800, 848, 3)
1278 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116851463670.png
(800, 848, 3)
1279 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116851497003.png
(800, 848, 3)
1280 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116851530336.png
(800, 848, 3)
1281 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116851563670.png
(800, 848, 3)
1

1347 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116853763547.png
(800, 848, 3)
1348 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116853796880.png
(800, 848, 3)
1349 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116853830213.png
(800, 848, 3)
1350 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116853863547.png
(800, 848, 3)
1351 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116853896880.png
(800, 848, 3)
1352 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116853930214.png
(800, 848, 3)
1353 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116853963547.png
(800, 848, 3)
1354 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116853996880.png
(800, 848, 3)
1355 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116854030214.png
(800, 848, 3)
1

1421 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116856230120.png
(800, 848, 3)
1422 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116856263483.png
(800, 848, 3)
1423 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116856296816.png
(800, 848, 3)
1424 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116856330150.png
(800, 848, 3)
1425 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116856363483.png
(800, 848, 3)
1426 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116856396816.png
(800, 848, 3)
1427 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116856430150.png
(800, 848, 3)
1428 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116856463483.png
(800, 848, 3)
1429 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116856496817.png
(800, 848, 3)
1

1495 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116858696703.png
(800, 848, 3)
1496 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116858730036.png
(800, 848, 3)
1497 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116858763353.png
(800, 848, 3)
1498 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116858796687.png
(800, 848, 3)
1499 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116858830020.png
(800, 848, 3)
1500 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116858863353.png
(800, 848, 3)
1501 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116858896687.png
(800, 848, 3)
1502 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116858930020.png
(800, 848, 3)
1503 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116858963354.png
(800, 848, 3)
1

1569 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116861173267.png
(800, 848, 3)
1570 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116861206600.png
(800, 848, 3)
1571 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116861239933.png
(800, 848, 3)
1572 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116861273246.png
(800, 848, 3)
1573 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116861296580.png
(800, 848, 3)
1574 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116861329914.png
(800, 848, 3)
1575 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116861373246.png
(800, 848, 3)
1576 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116861406579.png
(800, 848, 3)
1577 / 1591
/home/pang/data/dataset/iros2019slam/office/office-1-5/fisheye1/1560116861429914.png
(800, 848, 3)
1